In [50]:
%reload_ext autoreload
%autoreload 2
from airflow.hooks import S3Hook
s3_conn = S3Hook().get_conn()
from skills_ml.datasets.job_postings import job_postings, job_postings_chain
import random
import json
from skills_ml.algorithms.corpus_creators.basic import Doc2VecGensimCorpusCreator, CorpusCreator
from collections import Counter
import numpy as np
from skills_ml.algorithms.sampling.jobs import JobSampler

In [45]:
from skills_ml.algorithms.occupation_classifiers.base import VectorModel
embedding_model = VectorModel(s3_conn=s3_conn)

[2017-11-01 18:40:07,232] {utils.py:267} INFO - loading Doc2Vec object from tmp/gensim_doc2vec_va_0605
[2017-11-01 18:40:08,802] {utils.py:289} INFO - loading docvecs recursively from tmp/gensim_doc2vec_va_0605.docvecs.* with mmap=None
[2017-11-01 18:40:08,803] {utils.py:294} INFO - loading doctag_syn0 from tmp/gensim_doc2vec_va_0605.docvecs.doctag_syn0.npy with mmap=None
[2017-11-01 18:40:09,091] {utils.py:294} INFO - loading syn1 from tmp/gensim_doc2vec_va_0605.syn1.npy with mmap=None
[2017-11-01 18:40:09,320] {utils.py:294} INFO - loading syn0 from tmp/gensim_doc2vec_va_0605.syn0.npy with mmap=None
[2017-11-01 18:40:09,629] {utils.py:326} INFO - setting ignored attribute syn0norm to None
[2017-11-01 18:40:09,631] {utils.py:326} INFO - setting ignored attribute cum_table to None
[2017-11-01 18:40:09,632] {utils.py:273} INFO - loaded tmp/gensim_doc2vec_va_0605


# Streaming Data to Create Corpus

## Simple Corpus

In [55]:
job_postings_generator = job_postings_chain(s3_conn, ['2011Q2'], 'open-skills-private/test_corpus')
corpus = CorpusCreator(job_postings_generator)

In [56]:
corpus = list(corpus)

[2017-11-01 18:41:27,681] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/
[2017-11-01 18:41:27,752] {s3.py:83} INFO - 0 bytes transferred out of 0 total
[2017-11-01 18:41:27,754] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/CB_6874581a-5942-48fd-ba7e-2b1cdc38563f
[2017-11-01 18:41:27,823] {s3.py:83} INFO - 0 bytes transferred out of 18588132 total
[2017-11-01 18:41:28,485] {s3.py:83} INFO - 2072576 bytes transferred out of 18588132 total
[2017-11-01 18:41:28,724] {s3.py:83} INFO - 4145152 bytes transferred out of 18588132 total
[2017-11-01 18:41:28,922] {s3.py:83} INFO - 6217728 bytes transferred out of 18588132 total
[2017-11-01 18:41:29,082] {s3.py:83} INFO - 8290304 bytes transferred out of 18588132 total
[2017-11-01 18:41:29,237] {s3.py:83} INFO - 10362880 bytes transferred out of 18588132 total
[2017-11-01 18:41:29,411] {s3.py:83} INFO - 12435456 bytes transferred out of 18588132 total
[2017-11-01 18:41:29,551] {s

/Users/Eddie/Documents/DSSG/WDI/skills-ml/venv/lib/python3.5/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


[2017-11-01 18:41:41,622] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/NLX_b8384025-fa09-417d-ae10-96880fac86be
[2017-11-01 18:41:42,604] {s3.py:83} INFO - 0 bytes transferred out of 20171705 total
[2017-11-01 18:41:43,217] {s3.py:83} INFO - 2244608 bytes transferred out of 20171705 total
[2017-11-01 18:41:43,369] {s3.py:83} INFO - 4489216 bytes transferred out of 20171705 total
[2017-11-01 18:41:43,573] {s3.py:83} INFO - 6733824 bytes transferred out of 20171705 total
[2017-11-01 18:41:43,779] {s3.py:83} INFO - 8978432 bytes transferred out of 20171705 total
[2017-11-01 18:41:43,933] {s3.py:83} INFO - 11223040 bytes transferred out of 20171705 total
[2017-11-01 18:41:44,121] {s3.py:83} INFO - 13467648 bytes transferred out of 20171705 total
[2017-11-01 18:41:44,316] {s3.py:83} INFO - 15712256 bytes transferred out of 20171705 total
[2017-11-01 18:41:44,550] {s3.py:83} INFO - 17956864 bytes transferred out of 20171705 total
[2017-11-01 18:41:44,728] {

In [57]:
len(corpus)

10567

In [58]:
corpus[0]

{'@context': 'http://schema.org',
 '@type': 'JobPosting',
 'alternateName': 'Payroll Specialist',
 'baseSalary': {'@type': 'MonetaryAmount', 'maxValue': 0.0, 'minValue': 0.0},
 'datePosted': '2012-06-20',
 'description': 'Position Summary: The Payroll Specialist is responsible for processing all payroll information for accurate and timely payroll distribution. Responsibilities: Process store-level biweekly payroll accurately and timely Import store employee info and time clock data Research and resolve employee payroll issues Process involuntary wage withholding correspondence Respond promptly to all requests and communications received Review and process adjustments to hours/earnings Maintain employee direct deposit records Assist with various projects as assigned GREAT OPPORTUNITY!',
 'educationRequirements': 'Graduate Degree',
 'employmentType': 'Full-Time',
 'experienceRequirements': 'Qualifications: 1-3 years of general payroll experience Positive attitude combined with excellent 

## Creating Corpus with Criteria

#### One can define their own funciton and logic of filtering based on the common schema.

In [59]:
def major_group_filter_func(document):
    if document['onet_soc_code']:
        if document['onet_soc_code'][:2] in ['11', '13']:
            return document

def full_soc_code_filter_func(document):
    if document['onet_soc_code']:
        if document['onet_soc_code'] in ['11-9051.00', '17-3026.00']:
            return document

def wage_filter_func(document):
    if document['baseSalary']['minValue']:
        if float(document['baseSalary']['minValue']) >= 60000.0:
            return document

### Filtered by Major Groups

In [60]:
job_postings_generator = job_postings_chain(s3_conn, ['2011Q2'], 'open-skills-private/test_corpus')
corpus = CorpusCreator(job_postings_generator, filter_func=major_group_filter_func)

In [61]:
corpus = list(corpus)

[2017-11-01 18:42:03,267] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/
[2017-11-01 18:42:03,335] {s3.py:83} INFO - 0 bytes transferred out of 0 total
[2017-11-01 18:42:03,337] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/CB_6874581a-5942-48fd-ba7e-2b1cdc38563f
[2017-11-01 18:42:03,415] {s3.py:83} INFO - 0 bytes transferred out of 18588132 total
[2017-11-01 18:42:04,050] {s3.py:83} INFO - 2072576 bytes transferred out of 18588132 total
[2017-11-01 18:42:04,337] {s3.py:83} INFO - 4145152 bytes transferred out of 18588132 total
[2017-11-01 18:42:04,597] {s3.py:83} INFO - 6217728 bytes transferred out of 18588132 total
[2017-11-01 18:42:04,743] {s3.py:83} INFO - 8290304 bytes transferred out of 18588132 total
[2017-11-01 18:42:04,920] {s3.py:83} INFO - 10362880 bytes transferred out of 18588132 total
[2017-11-01 18:42:05,132] {s3.py:83} INFO - 12435456 bytes transferred out of 18588132 total
[2017-11-01 18:42:05,377] {s

/Users/Eddie/Documents/DSSG/WDI/skills-ml/venv/lib/python3.5/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


[2017-11-01 18:42:06,705] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/NLX_b8384025-fa09-417d-ae10-96880fac86be
[2017-11-01 18:42:06,880] {s3.py:83} INFO - 0 bytes transferred out of 20171705 total
[2017-11-01 18:42:07,706] {s3.py:83} INFO - 2244608 bytes transferred out of 20171705 total
[2017-11-01 18:42:08,234] {s3.py:83} INFO - 4489216 bytes transferred out of 20171705 total
[2017-11-01 18:42:08,757] {s3.py:83} INFO - 6733824 bytes transferred out of 20171705 total
[2017-11-01 18:42:09,292] {s3.py:83} INFO - 8978432 bytes transferred out of 20171705 total
[2017-11-01 18:42:09,810] {s3.py:83} INFO - 11223040 bytes transferred out of 20171705 total
[2017-11-01 18:42:10,419] {s3.py:83} INFO - 13467648 bytes transferred out of 20171705 total
[2017-11-01 18:42:10,926] {s3.py:83} INFO - 15712256 bytes transferred out of 20171705 total
[2017-11-01 18:42:11,343] {s3.py:83} INFO - 17956864 bytes transferred out of 20171705 total
[2017-11-01 18:42:11,646] {

In [65]:
corpus[0]

{'@context': 'http://schema.org',
 '@type': 'JobPosting',
 'alternateName': 'General Manager',
 'baseSalary': {'@type': 'MonetaryAmount', 'maxValue': 0.0, 'minValue': 0.0},
 'datePosted': '2012-09-17',
 'description': 'If you are a personable, outgoing and service-oriented professional who wants to manage a dynamic office, come join the Kool Smiles family! Kool Smiles was founded in 2002 and was built on the belief that every family has the right to quality dental care in a clean, safe and fun environment. Our first goal is to provide quality and compliant dental care to individuals in communities that typically get overlooked by other dentists. Our services aren’t just for kids- we know that children are more likely to have great dental habits if mom and dad do too, so we happily care for the entire family. Kool Smiles is looking for a customer-service-oriented leader to fill the role of Office Manager. As the Office Manager, you will manage an office staff of 15 to 30 employees and p

In [66]:
major_group = list(map(lambda c: c['onet_soc_code'][:2], corpus))

In [67]:
Counter(major_group)

Counter({'11': 283, '13': 102})

### Filtered by Full O*NET SOC Code

In [68]:
job_postings_generator = job_postings_chain(s3_conn, ['2011Q2'], 'open-skills-private/test_corpus')
corpus = CorpusCreator(job_postings_generator, filter_func=full_soc_code_filter_func)

In [69]:
corpus = list(corpus)

[2017-11-01 18:54:59,095] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/
[2017-11-01 18:54:59,188] {s3.py:83} INFO - 0 bytes transferred out of 0 total
[2017-11-01 18:54:59,190] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/CB_6874581a-5942-48fd-ba7e-2b1cdc38563f
[2017-11-01 18:54:59,376] {s3.py:83} INFO - 0 bytes transferred out of 18588132 total
[2017-11-01 18:55:00,001] {s3.py:83} INFO - 2072576 bytes transferred out of 18588132 total
[2017-11-01 18:55:00,263] {s3.py:83} INFO - 4145152 bytes transferred out of 18588132 total
[2017-11-01 18:55:00,439] {s3.py:83} INFO - 6217728 bytes transferred out of 18588132 total
[2017-11-01 18:55:00,634] {s3.py:83} INFO - 8290304 bytes transferred out of 18588132 total
[2017-11-01 18:55:00,854] {s3.py:83} INFO - 10362880 bytes transferred out of 18588132 total
[2017-11-01 18:55:01,078] {s3.py:83} INFO - 12435456 bytes transferred out of 18588132 total
[2017-11-01 18:55:01,310] {s

/Users/Eddie/Documents/DSSG/WDI/skills-ml/venv/lib/python3.5/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


[2017-11-01 18:55:02,329] {s3.py:83} INFO - 2244608 bytes transferred out of 20171705 total
[2017-11-01 18:55:02,494] {s3.py:83} INFO - 4489216 bytes transferred out of 20171705 total
[2017-11-01 18:55:02,683] {s3.py:83} INFO - 6733824 bytes transferred out of 20171705 total
[2017-11-01 18:55:02,843] {s3.py:83} INFO - 8978432 bytes transferred out of 20171705 total
[2017-11-01 18:55:03,002] {s3.py:83} INFO - 11223040 bytes transferred out of 20171705 total
[2017-11-01 18:55:03,168] {s3.py:83} INFO - 13467648 bytes transferred out of 20171705 total
[2017-11-01 18:55:03,316] {s3.py:83} INFO - 15712256 bytes transferred out of 20171705 total
[2017-11-01 18:55:03,446] {s3.py:83} INFO - 17956864 bytes transferred out of 20171705 total
[2017-11-01 18:55:03,609] {s3.py:83} INFO - 20171705 bytes transferred out of 20171705 total
[2017-11-01 18:55:03,963] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/VA_c3b9dad2-ef6c-4439-a4e7-6dff0a55d362
[2017-11-01 18:55:04,

In [70]:
soc = list(map(lambda c: c['onet_soc_code'], corpus))

In [71]:
Counter(soc)

Counter({'11-9051.00': 79, '17-3026.00': 65})

### Filtered by Minimum Wage >= 60000

In [72]:
job_postings_generator = job_postings_chain(s3_conn, ['2011Q2'], 'open-skills-private/test_corpus')
corpus = CorpusCreator(job_postings_generator, filter_func=wage_filter_func)

In [73]:
corpus = list(corpus)

[2017-11-01 18:55:04,756] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/
[2017-11-01 18:55:04,860] {s3.py:83} INFO - 0 bytes transferred out of 0 total
[2017-11-01 18:55:04,862] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/CB_6874581a-5942-48fd-ba7e-2b1cdc38563f
[2017-11-01 18:55:05,036] {s3.py:83} INFO - 0 bytes transferred out of 18588132 total
[2017-11-01 18:55:05,415] {s3.py:83} INFO - 2072576 bytes transferred out of 18588132 total
[2017-11-01 18:55:05,572] {s3.py:83} INFO - 4145152 bytes transferred out of 18588132 total
[2017-11-01 18:55:05,726] {s3.py:83} INFO - 6217728 bytes transferred out of 18588132 total
[2017-11-01 18:55:05,896] {s3.py:83} INFO - 8290304 bytes transferred out of 18588132 total
[2017-11-01 18:55:06,105] {s3.py:83} INFO - 10362880 bytes transferred out of 18588132 total
[2017-11-01 18:55:06,280] {s3.py:83} INFO - 12435456 bytes transferred out of 18588132 total
[2017-11-01 18:55:06,410] {s

In [74]:
min_wage = list(map(lambda c: c['baseSalary']['minValue'], corpus))

In [75]:
Counter(min_wage)

Counter({60000.0: 8,
         65000.0: 10,
         70000.0: 4,
         71110: 1,
         75000.0: 5,
         80000.0: 10,
         83221: 1,
         85000.0: 1,
         90000.0: 12,
         100000.0: 2,
         120000.0: 3,
         160000.0: 1})

# Sampling from Corpus

## Sampling from simple corpus

In [76]:
job_postings_generator = job_postings_chain(s3_conn, ['2011Q2'], 'open-skills-private/test_corpus')
corpus = CorpusCreator(job_postings_generator)

In [77]:
from skills_ml.algorithms.sampling.jobs import JobSampler
job_sampler = JobSampler(corpus, random_state=42)
corpus = job_sampler.sample(100)

[2017-11-01 19:38:58,303] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/
[2017-11-01 19:38:58,479] {s3.py:83} INFO - 0 bytes transferred out of 0 total
[2017-11-01 19:38:58,481] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/CB_6874581a-5942-48fd-ba7e-2b1cdc38563f
[2017-11-01 19:38:58,606] {s3.py:83} INFO - 0 bytes transferred out of 18588132 total
[2017-11-01 19:38:59,256] {s3.py:83} INFO - 2072576 bytes transferred out of 18588132 total
[2017-11-01 19:38:59,410] {s3.py:83} INFO - 4145152 bytes transferred out of 18588132 total
[2017-11-01 19:38:59,567] {s3.py:83} INFO - 6217728 bytes transferred out of 18588132 total
[2017-11-01 19:38:59,705] {s3.py:83} INFO - 8290304 bytes transferred out of 18588132 total
[2017-11-01 19:38:59,936] {s3.py:83} INFO - 10362880 bytes transferred out of 18588132 total
[2017-11-01 19:39:00,113] {s3.py:83} INFO - 12435456 bytes transferred out of 18588132 total
[2017-11-01 19:39:00,258] {s

/Users/Eddie/Documents/DSSG/WDI/skills-ml/venv/lib/python3.5/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


[2017-11-01 19:39:12,342] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/NLX_b8384025-fa09-417d-ae10-96880fac86be
[2017-11-01 19:39:13,912] {s3.py:83} INFO - 0 bytes transferred out of 20171705 total
[2017-11-01 19:39:14,531] {s3.py:83} INFO - 2244608 bytes transferred out of 20171705 total
[2017-11-01 19:39:14,693] {s3.py:83} INFO - 4489216 bytes transferred out of 20171705 total
[2017-11-01 19:39:14,832] {s3.py:83} INFO - 6733824 bytes transferred out of 20171705 total
[2017-11-01 19:39:15,027] {s3.py:83} INFO - 8978432 bytes transferred out of 20171705 total
[2017-11-01 19:39:15,169] {s3.py:83} INFO - 11223040 bytes transferred out of 20171705 total
[2017-11-01 19:39:15,318] {s3.py:83} INFO - 13467648 bytes transferred out of 20171705 total
[2017-11-01 19:39:15,482] {s3.py:83} INFO - 15712256 bytes transferred out of 20171705 total
[2017-11-01 19:39:15,672] {s3.py:83} INFO - 17956864 bytes transferred out of 20171705 total
[2017-11-01 19:39:15,842] {

In [78]:
corpus[0]

{'@context': 'http://schema.org',
 '@type': 'JobPosting',
 'alternateName': 'Customer Service Representative',
 'baseSalary': {'@type': 'MonetaryAmount', 'maxValue': 11.0, 'minValue': 11.0},
 'datePosted': '2011-08-25',
 'description': 'We are currently looking for bilingual (Spanish- English) customer service representative. You need to be fluent in both languages, clerical experience, be customer service oriented and have a professional appearance. If you think you have great customer service skills, have a strong work ethic and want to be part of a great team please contact me',
 'educationRequirements': 'Not Specified',
 'employmentType': 'Full-Time',
 'experienceRequirements': 'MINIMUM QUALIFICATIONS: High school diploma and some college coursework or equivalent Capable of solving a variety of customer service problems and supporting a range of personalities and customer types Bi-lingual: Spanish Ability to communicate effectively with others, orally and written',
 'id': 'CB_-1261

In [79]:
industry = list(map(lambda c: c['industry'], corpus))

In [80]:
Counter(industry)

Counter({'': 59,
         'Advertising, Sales - Marketing, Hospitality': 1,
         'Biotechnology, Pharmaceutical, Food': 1,
         'Consulting, Sales - Marketing, Hospitality': 1,
         'Consumer Products, Other, Food': 1,
         'Consumer Products, Sales - Marketing, Food': 2,
         'Education - Teaching - Administration, Social Services, Food': 1,
         'Entertainment, Food, Hospitality': 1,
         'Entertainment, Retail, Hospitality': 1,
         'Hospitality, Restaurant, Food': 4,
         'Not for Profit - Charitable, Healthcare - Health Services, Food': 1,
         'Packaging, Manufacturing, Food': 1,
         'Public Relations, Not for Profit - Charitable, Food': 1,
         'Restaurant, Food, Hospitality': 7,
         'Restaurant, Full Service Restaurant, Hospitality': 1,
         'Restaurant, Managed Care, Hospitality': 1,
         'Restaurant, Retail, Hospitality': 6,
         'Retail, Restaurant, Food': 1,
         'Sales - Marketing, Consulting, Hospitalit

## Sampling from filtered corpus

### Reservoir Sampling

In [42]:
job_postings_generator = job_postings_chain(s3_conn, ['2011Q2'], 'open-skills-private/test_corpus')
corpus = CorpusCreator(job_postings_generator, filter_func=major_group_filter_func)

In [43]:
job_sampler = JobSampler(corpus, random_state=42)
corpus = job_sampler.sample(100)

[2017-11-01 18:07:50,774] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/
[2017-11-01 18:07:50,863] {s3.py:83} INFO - 0 bytes transferred out of 0 total
[2017-11-01 18:07:50,865] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/CB_6874581a-5942-48fd-ba7e-2b1cdc38563f
[2017-11-01 18:07:51,009] {s3.py:83} INFO - 0 bytes transferred out of 18588132 total
[2017-11-01 18:07:51,582] {s3.py:83} INFO - 2072576 bytes transferred out of 18588132 total
[2017-11-01 18:07:51,815] {s3.py:83} INFO - 4145152 bytes transferred out of 18588132 total
[2017-11-01 18:07:51,961] {s3.py:83} INFO - 6217728 bytes transferred out of 18588132 total
[2017-11-01 18:07:52,110] {s3.py:83} INFO - 8290304 bytes transferred out of 18588132 total
[2017-11-01 18:07:52,252] {s3.py:83} INFO - 10362880 bytes transferred out of 18588132 total
[2017-11-01 18:07:52,387] {s3.py:83} INFO - 12435456 bytes transferred out of 18588132 total
[2017-11-01 18:07:52,525] {s

In [34]:
onet_soc_code = list(map(lambda c: c['onet_soc_code'][:2], corpus))

In [35]:
Counter(onet_soc_code)

Counter({'11': 78, '13': 22})

### Weighted Reservoir Sampling

In [93]:
job_postings_generator = job_postings_chain(s3_conn, ['2011Q2'], 'open-skills-private/test_corpus')
corpus = CorpusCreator(job_postings_generator, filter_func=major_group_filter_func)

In [94]:
job_sampler = JobSampler(corpus, major_group=True,weights={'11': 1, '13': 3.5})
sampled_corpus = job_sampler.sample(50)

[2017-11-01 18:18:43,238] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/
[2017-11-01 18:18:43,329] {s3.py:83} INFO - 0 bytes transferred out of 0 total
[2017-11-01 18:18:43,331] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/CB_6874581a-5942-48fd-ba7e-2b1cdc38563f
[2017-11-01 18:18:43,396] {s3.py:83} INFO - 0 bytes transferred out of 18588132 total
[2017-11-01 18:18:44,142] {s3.py:83} INFO - 2072576 bytes transferred out of 18588132 total
[2017-11-01 18:18:44,346] {s3.py:83} INFO - 4145152 bytes transferred out of 18588132 total
[2017-11-01 18:18:44,501] {s3.py:83} INFO - 6217728 bytes transferred out of 18588132 total
[2017-11-01 18:18:44,682] {s3.py:83} INFO - 8290304 bytes transferred out of 18588132 total
[2017-11-01 18:18:44,813] {s3.py:83} INFO - 10362880 bytes transferred out of 18588132 total
[2017-11-01 18:18:44,956] {s3.py:83} INFO - 12435456 bytes transferred out of 18588132 total
[2017-11-01 18:18:45,089] {s

In [95]:
major_group = list(map(lambda c: c[1][:2], sampled_corpus))

In [96]:
Counter(major_group)

Counter({'11': 24, '13': 26})

In [97]:
sampled_corpus[0]

({'@context': 'http://schema.org',
  '@type': 'JobPosting',
  'alternateName': 'General Manager',
  'baseSalary': {'@type': 'MonetaryAmount',
   'maxValue': 27500.0,
   'minValue': 25000.0},
  'datePosted': '2012-07-19',
  'description': 'Restaurant General Manager Average bonus $7,000 - $10,000 annually! Health Care Company paid cell phone One week vacation per year Restaurant General Manager Responsibilities * Overall Restaurant operations of fast casual restaurant * Appoint tasks and responsibilities to other managers and team members * Restaurant General Manager will ensure policies are upheld and utilize corrective action when necessary * Manage financial forecasting (P&L, Sales, Budgets, etc.) * Train, coach and hold regular meetings with team members * Other typical responsibilities to ensure ease of operations',
  'educationRequirements': 'Graduate Degree',
  'employmentType': 'Full-Time',
  'experienceRequirements': '1. Restaurant General Manager should have one year of superv